In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import conj
from src.zmatrix_models import MetodoMatrizZdetailed

def carregar_dados_sistema():
    # Carregar dados das cargas
    spot_loads = pd.read_excel('../feeder13/Spot Load Data.xls')
    
    # Carregar dados das linhas
    line_data = pd.read_excel('../feeder13/Line Data.xls')
    
    return spot_loads, line_data

def sistema_13_barras():
    # Carregar dados do sistema
    spot_loads, line_data = carregar_dados_sistema()
    
    # Definir o sistema
    barras = list(range(1, 14))  # Barras de 1 a 13S
    
    # Barra 1 é a slack (tipo 0)
    # Todas as outras barras são PQ (tipo 2)
    barras_tipo = {1: 0}  # Barra slack
    for b in range(2, 14):
        barras_tipo[b] = 2  # Barras PQ
    
    # Potências complexas (S = P + jQ) em pu
    # Converter cargas de kW/kvar para pu (base de 1000 kVA)
    base_power = 1000  # kVA
    potencias = {1: 0.0 + 0.0j}  # Barra slack
    
    for _, row in spot_loads.iterrows():
        barra = int(row['Bus'])
        p = -row['kW'] / base_power  # Negativo pois é carga
        q = -row['kvar'] / base_power  # Negativo pois é carga
        potencias[barra] = p + 1j*q
    
    # Tensões iniciais (flat start com valores de 1.0 pu, exceto a barra slack)
    v_inicial = {1: 1.0 + 0.0j}  # Barra slack
    for b in range(2, 14):
        v_inicial[b] = 1.0 + 0.0j
    
    # Linhas (conexões entre barras)
    linhas = []
    z_linhas = {}
    
    for _, row in line_data.iterrows():
        de = int(row['From Bus'])
        para = int(row['To Bus'])
        linhas.append((de, para))
        
        # Impedância da linha em pu (base de 1000 kVA)
        r = row['R (ohms)'] / 100  # Converter para pu
        x = row['X (ohms)'] / 100  # Converter para pu
        z_linhas[(de, para)] = r + 1j*x
    
    # Admitância shunt total por linha (j0.01 pu)
    y_shunt = 0.01j
    
    # Construir a matriz de admitância Y_bus
    n_barras = len(barras)
    Y_bus = np.zeros((n_barras, n_barras), dtype=complex)
    
    # Preencher a matriz Y com os elementos fora da diagonal principal
    for (de, para) in linhas:
        idx_de = barras.index(de)
        idx_para = barras.index(para)
        
        # Admitância série da linha
        y_serie = 1.0 / z_linhas[(de, para)]
        
        # Elementos fora da diagonal são negativos da admitância série
        Y_bus[idx_de, idx_para] = -y_serie
        Y_bus[idx_para, idx_de] = -y_serie
        
        # Adicionar metade da admitância shunt às barras em cada extremidade
        Y_bus[idx_de, idx_de] += y_serie + y_shunt/2
        Y_bus[idx_para, idx_para] += y_serie + y_shunt/2
    
    # Tolerância para convergência
    tolerancia = 0.0010
    
    # Criar instância do solver
    solver = MetodoMatrizZdetailed(Y_bus, barras, linhas, barras_tipo, potencias, v_inicial, tol=tolerancia)
    
    # Resolver usando método da matriz Z com substituição em bloco
    v_bloco, v_history_bloco, iteracoes_bloco = solver.substituicao_em_bloco()
    fluxos_bloco = solver.calcular_fluxos(v_bloco)
    
    # Exibir resultados
    solver.exibir_resultados(v_bloco, fluxos_bloco, "Substituição em Bloco", iteracoes_bloco)
    solver.plotar_convergencia(v_history_bloco, "Substituição em Bloco")
    
    # Resolver usando método da matriz Z com substituição direta
    v_direta, v_history_direta, iteracoes_direta = solver.substituicao_direta()
    fluxos_direta = solver.calcular_fluxos(v_direta)
    
    # Exibir resultados
    solver.exibir_resultados(v_direta, fluxos_direta, "Substituição Direta", iteracoes_direta)
    solver.plotar_convergencia(v_history_direta, "Substituição Direta")
    
    # Comparar número de iterações
    print(f"\nComparação do número de iterações:")
    print(f"Método de Substituição em Bloco: {iteracoes_bloco} iterações")
    print(f"Método de Substituição Direta: {iteracoes_direta} iterações")

if __name__ == "__main__":
    sistema_13_barras()

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.